In [ ]:
%pip install --quiet bs4 newspaper3k lxml_html_clean python-dotenv pymysql mysql-connector-python sqlalchemy pandas

%pip install langchain_text_splitters langchain-community langchain langchain-chroma langchain-teddynote langchain-openai

In [114]:
import bs4
from langchain import hub
from langchain_community.llms import GPT4All
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from newspaper import Article
from bs4 import BeautifulSoup
from langchain.document_loaders import WebBaseLoader
from langchain.schema import Document
from sqlalchemy import create_engine, MetaData, Table, select, Column, Integer, String, Text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from dotenv import load_dotenv
import os
import requests


In [474]:
load_dotenv()

CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_USERNAME = os.getenv("DB_USERNAME")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEME = os.getenv("DB_SCHEME")

In [116]:
# collection_name = 'chroma_stock_news'
collection_name = 'chroma_stock_news_v2'


In [505]:
import bs4
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


def process_news_data_to_document(news_data):
  documents = []

  for index, row in news_data.iterrows():
    title = row['TITLE']
    content = row['CONTENT']
    
    document = Document(
        page_content=content,
        metadata={
          "title": title,        
          "stock_name": stock_name,
          "main_category": main_category,
          "sub_category": sub_category}
    )
    documents.append(document)
  
  return documents

def chunk_documents(documents, chunk_size):
    for i in range(0, len(documents), chunk_size):
      yield documents[i:i + chunk_size]

def embed_news_data(documents, text_splitter):

  batch_size = 5461
  splits = text_splitter.split_documents(documents)

  for doc_batch in chunk_documents(splits, batch_size):
    try:
      Chroma.from_documents(
        documents=doc_batch,
        embedding=OpenAIEmbeddings(model='text-embedding-3-large'),
        collection_name='chroma_stock_news_2',
        persist_directory="./chroma_3-large"
      )
    except Exception as e:
      print(f"Record Failed: {e}")

In [ ]:
persist_db = Chroma(
  embedding_function=OpenAIEmbeddings(model='text-embedding-3-large'),
  collection_name='chroma_stock_news',
  persist_directory="./chroma_3-large"
)

persist_db.get()

In [503]:
len(persist_db.get()['ids'])

228842

In [499]:
persist_db.get('6f61f9cc-f45d-4d67-a55b-6a8878e41eaf') 

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [481]:

def search_naver_news(query, display=10, start=1, sort='sim'):
    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }
    params = {
        "query": query,  # 검색어
        "display": display,  # 가져올 결과 수
        "start": start,  # 검색 시작 위치
        "sort": sort  # 정렬 기준: date(날짜순), sim(유사도순)
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json() 
    else:
        print("Error:", response.status_code)
        return None

In [239]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
RAG


In [482]:
engine = create_engine(f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_SCHEME}')

Session = sessionmaker(bind=engine)
session = Session()

metadata = MetaData()

# 테이블 정의
stock_table = Table('STOCK', metadata, autoload_with=engine)
sub_category_table = Table('SUB_CATEGORY', metadata, autoload_with=engine)
main_category_table = Table('MAIN_CATEGORY', metadata, autoload_with=engine)

# 쿼리 작성
stmt = select(
    stock_table.c.STOCK_ID,
    stock_table.c.STOCK_NAME,
    sub_category_table.c.SUB_CATEGORY_NAME,
    main_category_table.c.MAIN_CATEGORY_NAME
).join(
    sub_category_table, stock_table.c.SUB_CATEGORY_ID == sub_category_table.c.SUB_CATEGORY_ID
).join(
    main_category_table, sub_category_table.c.MAIN_CATEGORY_ID == main_category_table.c.MAIN_CATEGORY_ID
)

# 쿼리 실행
result = session.execute(stmt)

# 결과를 리스트로 변환
stock_list = [(row[0], row[1], row[2], row[3]) for row in result]

# 세션 종료
session.close()

# 결과 출력
print(stock_list)


[(28244, '한국테크놀로지', '4차 산업', '숙박 및 음식'), (29327, '동아타이어', '4차 산업', '숙박 및 음식'), (10763, 'Test Stock', 'IT기기', '숙박 및 음식'), (27524, '서부T&D', '숙박·음식', '숙박 및 음식'), (27880, '아난티', '숙박·음식', '숙박 및 음식'), (29126, '디딤이앤에프', '숙박·음식', '숙박 및 음식'), (27325, '한탑', '음식료·담배', '숙박 및 음식'), (27365, '대주산업', '음식료·담배', '숙박 및 음식'), (27428, '창해에탄올', '음식료·담배', '숙박 및 음식'), (27474, '푸드웰', '음식료·담배', '숙박 및 음식'), (27488, '한일사료', '음식료·담배', '숙박 및 음식'), (27498, '매일홀딩스', '음식료·담배', '숙박 및 음식'), (27616, '엠에스씨', '음식료·담배', '숙박 및 음식'), (27756, '현대사료', '음식료·담배', '숙박 및 음식'), (27779, '진로발효', '음식료·담배', '숙박 및 음식'), (27842, '풍국주정', '음식료·담배', '숙박 및 음식'), (27876, '케이씨피드', '음식료·담배', '숙박 및 음식'), (27892, '팜스토리', '음식료·담배', '숙박 및 음식'), (28001, '이지홀딩스', '음식료·담배', '숙박 및 음식'), (28126, '국순당', '음식료·담배', '숙박 및 음식'), (28367, '체리부로', '음식료·담배', '숙박 및 음식'), (28452, '우리손에프앤지', '음식료·담배', '숙박 및 음식'), (28519, '우리바이오', '음식료·담배', '숙박 및 음식'), (28578, '동우팜투테이블', '음식료·담배', '숙박 및 음식'), (28616, '아미코젠', '음식료·담배', '숙박 및 음식'), (28728, '우양', '음식료·담배', '숙박 및 음식'), (

In [207]:
engine = create_engine(f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_SCHEME}')

Session = sessionmaker(bind=engine)
session = Session()

metadata = MetaData()

# 테이블 정의
user_table = Table('USER', metadata, autoload_with=engine)
ITA_table = Table('INVESTMENT_TYPE_ANSWER', metadata, autoload_with=engine)
IT_table = Table('INVESTMENT_TYPE', metadata, autoload_with=engine)
INTR_table = Table('INTEREST_CATEGORY', metadata, autoload_with=engine)
sub_category_table = Table('SUB_CATEGORY', metadata, autoload_with=engine)
main_category_table = Table('MAIN_CATEGORY', metadata, autoload_with=engine)
# 쿼리 작성
stmt = select(
    user_table.c.USER_ID,
    IT_table.c.INVESTMENT_TYPE_NAME,
    IT_table.c.CONTENT,
    IT_table.c.STOCK_ALLOCATION_CONTENT,
    IT_table.c.STOCK_SELECTION_CONTENT,
    IT_table.c.RISK_MANAGEMENT_CONTENT,
    main_category_table.c.MAIN_CATEGORY_NAME
).join(
    ITA_table, user_table.c.USER_ID == ITA_table.c.USER_ID
).join(
    IT_table, ITA_table.c.INVESTMENT_TYPE_ID == IT_table.c.INVESTMENT_TYPE_ID
).join(
    INTR_table, user_table.c.USER_ID == INTR_table.c.USER_ID
).join(
    sub_category_table, INTR_table.c.SUB_CATEGORY_ID == sub_category_table.c.SUB_CATEGORY_ID
).join(
    main_category_table, sub_category_table.c.MAIN_CATEGORY_ID == main_category_table.c.MAIN_CATEGORY_ID
)

# 쿼리 실행
result = session.execute(stmt)

# 결과를 리스트로 변환
recommend_list = [(row[0], row[1], row[2], row[3], row[4],row[5],row[6]) for row in result]

# 세션 종료
session.close()

# 결과 출력
print(recommend_list)

In [483]:
Base = declarative_base()

class News(Base):
    __tablename__ = 'NEWS'
    news_id = Column(Integer, primary_key=True, autoincrement=True)
    stock_id = Column(Integer, nullable=False)
    title = Column(String(20), nullable=True)
    content = Column(Text, nullable=True)
    link = Column(String(100), nullable=True)
    image = Column(String(100), nullable=True)

engine = create_engine(f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_SCHEME}')

Session = sessionmaker(bind=engine)
session = Session()

metadata = MetaData()

def save_news_to_database(news_data):
    try:
        for news_item in news_data:
            data = News(
                stock_id=news_item['stock_id'],
                title=news_item['title'],
                link=news_item['link'],
                content=news_item['content'],
                image=news_item['image']
            )
            session.add(data)

        session.commit()
    except Exception as e:
        session.rollback() 
        print(f"DB error occurred: {e}")


C:\Users\student\AppData\Local\Temp\ipykernel_18884\1720991271.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [484]:

def fetch_and_prepare_news_data(news_data_list):
  result_list = []
  docs = []
  for news_item in news_data_list:
    stock_id = news_item['stock_id']
    stock_name = news_item['stock_name']
    sub_category_name = news_item['sub_category_name']
    main_category_name =news_item['main_category_name'] 
    for news_url in news_item['news_url_list']:
      if (news_url.startswith("https://n.news.naver.com") == False):
        continue
      response = requests.get(news_url)
      html_content = response.content

      soup = BeautifulSoup(html_content, 'html.parser')

      title_element = soup.find("div", class_="media_end_head_title")
      content_element = soup.find("div", class_="newsct_article _article_body")

      if title_element is None or content_element is None:
          continue

      title = title_element.get_text(strip=True)
      content = content_element.get_text(strip=True)

      if not title or not content:
          continue
      
      contents_div = soup.find("div", id="contents")
      image_tag = contents_div.find("img")
      if image_tag:
        image_src = image_tag.get('src') or image_tag.get('data-src')

      result_list.append({
        'stock_id': stock_id,
        'title': title,
        'content': content,
        'link': news_url,
        # 'publish_date': publish_date,
        'image': image_src
      })

      document = Document(
        page_content=content,
        metadata={"title": title, "stock_name" : stock_name, "sub_category" : sub_category_name, "main_category" : main_category_name}
      )
      docs.append(document)

  return {"result_list": result_list, "docs": docs}
    # else:
    #   article = Article(news_url, language = 'ko') 

    #   article.download()
    #   article.parse()
    #   title = article.title
    #   text = article.text
    #   date = article.publish_date
    #   image_url = article.top_image

    #   doc_content = title + "\n" + text
    #   doc = Document(page_content=doc_content)
    #   docs.append(doc)

In [507]:

def process_and_store_news(stock_list):
  news_data_batch = []
  result_list = []
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

  for stock_id, stock_name, sub_category_name, main_category_name in stock_list:
    # 네이버 뉴스에서 주식 종목명을 검색하여 결과를 가져옴
    result = search_naver_news(stock_name)
    news_url_list = []
    if result:
      for idx, item in enumerate(result['items']):
        news_url_list.append(item['link'])
    
    # 수집된 뉴스 링크를 news_data_list에 추가
    news_data_batch.append({
      'stock_id' : stock_id,
      'stock_name' : stock_name,
      'sub_category_name' : sub_category_name,
      'main_category_name' : main_category_name,
      'news_url_list' : news_url_list
    })

    # 뉴스 종목 검색 갯수가 30개 이상일 경우 데이터를 처리
    if(len(news_data_batch) >= 30):

      # 뉴스 데이터 수집 및 문서로 변환
      news_data_docs_and_list = fetch_and_prepare_news_data(news_data_batch)
      # 결과 리스트와 문서 리스트를 가져옴
      news_to_save = news_data_docs_and_list['result_list']
      docs = news_data_docs_and_list['docs']

      print(f"{len(news_to_save)} jobs processing...")
      # 결과 리스트를 데이터베이스에 저장
      save_news_to_database(news_to_save)
      # 문서 데이터를 임베딩하여 처리
      embed_news_data(docs, text_splitter)
      
      print("save success")
      news_to_save = []
      docs = []
      news_data_batch=[]

process_and_store_news(stock_list)

Error: 429
Error: 429
Error: 429
67 jobs processing...
DB error occurred: (pymysql.err.OperationalError) (2006, "MySQL server has gone away (ConnectionAbortedError(10053, '현재 연결은 사용자의 호스트 시스템의 소프트웨어의 의해 중단되었습니다', None, 10053, None))")
[SQL: INSERT INTO `NEWS` (stock_id, title, content, link, image) VALUES (%(stock_id)s, %(title)s, %(content)s, %(link)s, %(image)s)]
[parameters: {'stock_id': 28244, 'title': "뷰런테크놀로지, '스마트 인프라 라이다 솔루션' GS인증 1등급 획득", 'content': '"기술 신뢰성 강화로 스마트 인프라 시장 선점 기대”뷰런테크놀로지(이하 뷰런)가 개발한 \'스마트 인프라 라이다 솔루션 V1.0\'이 한국산업기술시험원(KTL)에서 GS 인증 최고 등급인 1등급을 획득했다고 15일 밝혔다.GS 인증은 ISO 국제표준을 기준으로 소프트 ... (567 characters truncated) ...  새로운 기회를 창출할 수 있을 것”이라고 말했다.이어 “뷰런은 강점을 보유하고 있는 자율주행 솔루션에 더해 스마트 인프라 라이다 솔루션을 통해 지능형 교통 시스템, 교통관제 시스템 등 여러 분야에서 기술 활용성을 극대화해 스마트시티 구축에 앞장서고 있다”고 밝혔다.', 'link': 'https://n.news.naver.com/mnews/article/092/0002348699?sid=105', 'image': 'https://imgnews.pstatic.net/image/092/2024/10/15/0002348699_001_20241015171310035.jpg?type=w860'}]
(Background on th

In [ ]:
news_data_list = []

for stock_id, stock_name in stock_list:
  result = search_naver_news(stock_name)

  news_url_list = []
  all_splits = []

  if result:
    for idx, item in enumerate(result['items']):
      news_url_list.append(item['link'])

  news_data_list.append({
    'stock_id' : stock_id,
    'news_url_list' : news_url_list
  })
  
  if(len(news_data_list) >= 50):
    print("now saving...")
    result_list = []
    collect_news_data(news_data_list, all_splits, result_list)
    save_to_database(result_list)
    print(f'{len(result_list)} news saved')
    news_data_list = []


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

for news_url in news_data:
    article = Article(news_url, language = 'ko') 

    article.download()
    print(news_url)
    article.parse()
    title = article.title
    text = article.text
    date = article.publish_date
    image_url = article.top_image

    doc_content = title + "\n" + text
    doc = Document(page_content=doc_content)
    docs = [doc] 

    # add_to_verctor_db(text_splitter, docs)
    print(f"URL: {news_url}\n {title}\n {text}\n {date}\n{image_url}\n")

In [ ]:
all_splits = []


for news_url in news_data:
    if news_url.startswith("https://n.news.naver.com"):
        # 각 뉴스 URL에 대해 WebBaseLoader를 설정합니다.
        loader = WebBaseLoader(
            web_paths=(news_url,),
            bs_kwargs=dict(
                parse_only=bs4.SoupStrainer(
                    "div",
                    attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
                )
            ),
        )
        article = Article(news_url, language = 'ko') 

        article.download()
        article.parse()

        image_url = article.top_image
        docs = loader.load()
        print('docs :{docs}\n image_url :{image_url}')
    else:

        article = Article(news_url, language = 'ko') 

        article.download()
        article.parse()
        title = article.title
        text = article.text
        date = article.publish_date
        image_url = article.top_image

        doc_content = title + "\n" + text
        doc = Document(page_content=doc_content)
        docs = [doc] 

    print(f"URL: {news_url} {image_url}")
    
    splits = text_splitter.split_documents(docs)
    all_splits.extend(splits)


## OPENAI 모델

In [24]:
import pandas as pd
from sqlalchemy import create_engine

def fetchDBNews():

  # SQLAlchemy 엔진 생성 (MySQL 연결 문자열)
  engine = create_engine(f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_SCHEME}')

  # SQL 쿼리 실행하여 title과 content 가져오기
  query = "SELECT TITLE, CONTENT FROM NEWS"
  news_data = pd.read_sql(query, engine)

  # 데이터 미리 보기 (첫 5개의 데이터)
  return news_data

In [ ]:
['chroma_stock_news', 'chroma_stock_news_2']

In [553]:

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

llm = ChatOpenAI(model='gpt-4o')
# news_data = fetchDBNews()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

vectorstore = Chroma(collection_name='chroma_stock_news_2', persist_directory="./chroma_3-large", embedding_function=embedding)
# retriever = vectorstore.as_retriever(search_kwargs={'k': 7, 'filter': {'main_category':'숙박 및 음식'}})
# retriever = vectorstore.as_retriever()
retriever = vectorstore.as_retriever(search_kwargs={'k': 7})

system_prompt = (
"""당신은 최신 금융 동향을 분석하고 제공하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)을 바탕으로 금융 시장에 대한 동향을 전달하는 것입니다.
검색된 다음 문맥(context)을 사용하여 최신 금융 동향을 요약해 주세요.
만약, 주어진 문맥(context)에서 동향을 찾을 수 없다면, `주어진 정보에서 주식 종목에 대한 정보를 찾을 수 없습니다`라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#문맥(Context): 
{context} 

#한국 금융 시장 동향 요약:"""
)

# system_prompt = (
#   """당신은 주식 종목을 분석하고 제공하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)을 바탕으로 주식 종목에 대한 동향을 전달하는 것입니다.
# 검색된 다음 문맥(context)을 사용하여 최신 주식 종목을 요약해 주세요. 만약, 주어진 문맥(context)에서 동향을 찾을 수 없다면, `주어진 정보에서 주식 종목에 대한 정보를 찾을 수 없습니다`라고 답하세요.
# 한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.


# #문맥(Context): 
# {context} 

# #시장 동향 요약:"""
# )
# system_prompt = (
#   """당신은 주식 종목을 분석하고 제공하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)을 바탕으로 주식 종목에 대한 동향을 전달하는 것입니다.
# 최신 시장 데이터를 기반으로 단계별로 논리적인 사고 과정을 통해 분석을 진행하고 결론을 도출하세요. 검색된 다음 문맥(context)을 사용하여 최신 주식 종목을 요약해 주세요. 
# 만약, 주어진 문맥(context)에서 동향을 찾을 수 없다면, `0`라고 답하세요.
# 한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.
# 서론은 달지말고 본론부터 이야기해주세요.

# 더 깊게 생각해주세요.
# #문맥(Context): 
# {context} 

# #시장 동향 요약:"""
# )
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [554]:
trend_result = rag_chain.invoke({"input" : "최신 한국 금융 시장 동향"})

In [555]:
trend_result

{'input': '최신 한국 금융 시장 동향',
 'context': [Document(metadata={'main_category': '운송 및 물류', 'stock_name': 'HLB글로벌', 'sub_category': '유통업', 'title': '"한강 노벨문학상 수상, 삼성출판사·예스24 등 급등"'}, page_content="이들 기업의 주가 상승은 대부분 개인 투자자가 이끈 것으로 파악됩니다. 오늘도 일부 관련 종목들 급등하는 모습입니다. '테마주' 이기 때문에 매매에 유의하시는 것이 좋겠습니다.◇진행자> 최근 금융감독원에 따르면 외국인의 국내 주식 매도 규모가 코로나19 이후 최대치로 그 규모가 두 달 새 10조 원 정도라고 합니다. 금리 인하나 금융환경이 조금씩 풀릴 것이라고 생각되었는데요. 유독 한국 시장이 힘을 못 쓰는 것 같습니다. 왜 이렇게 파는 걸까요.◆조명은> 한국은행에서 발간한 자료에서는 AI 산업 성장성에 대한 불확실성과, 중동"),
  Document(metadata={'main_category': '미디어 서비스', 'stock_name': '삼성출판사', 'sub_category': '서비스업', 'title': '"한강 노벨문학상 수상, 삼성출판사·예스24 등 급등"'}, page_content="이들 기업의 주가 상승은 대부분 개인 투자자가 이끈 것으로 파악됩니다. 오늘도 일부 관련 종목들 급등하는 모습입니다. '테마주' 이기 때문에 매매에 유의하시는 것이 좋겠습니다.◇진행자> 최근 금융감독원에 따르면 외국인의 국내 주식 매도 규모가 코로나19 이후 최대치로 그 규모가 두 달 새 10조 원 정도라고 합니다. 금리 인하나 금융환경이 조금씩 풀릴 것이라고 생각되었는데요. 유독 한국 시장이 힘을 못 쓰는 것 같습니다. 왜 이렇게 파는 걸까요.◆조명은> 한국은행에서 발간한 자료에서는 AI 산업 성장성에 대한 불확실성과, 중동"),
  Document(metadata={'main_category': '금융 및 보험', 's

In [134]:
trend_system_prompt = (
"""당신은 최신 금융 동향을 분석하고 제공하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)을 바탕으로 금융 시장에 대한 동향을 전달하는 것입니다.
검색된 다음 문맥(context)을 사용하여 금융 시장 동향을 요약해 주세요. 메타데이터에 포함된 `main_category`를 기반으로 관련 내용을 전달해야 합니다.
만약, 주어진 문맥(context)에서 동향을 찾을 수 없다면, `주어진 정보에서 금융 시장 동향에 대한 정보를 찾을 수 없습니다`라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

메타데이터:
- 서브 카테고리 (main_category): {main_category}

#문맥(Context): 
{context} 

#금융 시장 동향 요약:"""
)

# 프롬프트에 필요한 요소를 준비하여 생성
trend_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", trend_system_prompt),
        ("human", "{input}"),
    ]
)
trend_question_answer_chain = create_stuff_documents_chain(llm, trend_prompt)
trend_rag_chain = create_retrieval_chain(retriever, trend_question_answer_chain)
stocK_trend = trend_rag_chain.invoke({"input": "입력된 메인 카테고리에 대한 동향", "main_category": "숙박 및 음식'에 대한 동향"})

In [ ]:
stocK_trend

In [21]:
from langchain_teddynote.messages import stream_response

In [ ]:
from langchain_core.prompts import PromptTemplate

# prompt = PromptTemplate.from_template(
#     """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
# 검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
# 한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

# #Question: 
# {question} 

# #Context: 
# {context} 

# #Answer:"""
# )

prompt = PromptTemplate.from_template(
    """당신은 최신 금융 동향을 분석하고 제공하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)을 바탕으로 금융 시장에 대한 동향을 전달하는 것입니다.
검색된 다음 문맥(context)을 사용하여 최신 금융 시장 동향을 요약해 주세요. 만약, 주어진 문맥(context)에서 동향을 찾을 수 없다면, `주어진 정보에서 금융 시장 동향에 대한 정보를 찾을 수 없습니다`라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#문맥(Context): 
{context} 

#금융 시장 동향 요약:"""
)

In [4]:
import pandas as pd
from sqlalchemy import create_engine

def fetchNews():
  engine = create_engine(f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_SCHEME}')

  query = "SELECT TITLE, CONTENT, SUB_CATEGORY FROM NEWS"
  news_data = pd.read_sql(query, engine)

  return news_data


In [5]:
import bs4
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

newsData = fetchNews()
documents = []

for index, row in newsData.iterrows():
    title = row['TITLE']
    content = row['CONTENT']
    
    document = Document(
        page_content=content,
        metadata={"title": title}
    )
    documents.append(document)

splits = text_splitter.split_documents(documents)
def chunk_documents(documents, chunk_size):
    for i in range(0, len(documents), chunk_size):
        yield documents[i:i + chunk_size]

# 최대 배치 크기를 5461로 설정
batch_size = 5461
splits = text_splitter.split_documents(documents)  # 문서 분리

# 각 배치별로 Chroma에 저장
for doc_batch in chunk_documents(splits, batch_size):
    vectorstore = Chroma.from_documents(
        documents=doc_batch,
        embedding=OpenAIEmbeddings(),
        collection_name='chroma_stock_news',
        persist_directory="./chroma"
    )

In [ ]:
%pip install scikit-learn

## 리포트 생성 프로세스
1. 임베딩
- 종목 2700개 조회
- 2700개에 대해 각 10개씩 뉴스 조회
- 뉴스를 네이버 뉴스만 추출
- 네이버 뉴스 크롤링
- 크롤링 데이터 임베딩
  - 메타데이터 : 종목 명, 메인 카테고리, 서브카테고리
2. 답변 생성
- 답변 준비 : 

  1. 시장 동향 답변 생성
  2. 종목 동향 답변 생성

  - 실패에 대한 처리
  1. k값 증가 후 재질의(k <= 200)
  2. 메타데이터 필터링 후 재질의(k <= 200)
  3. 벡터 DB 전환

In [710]:

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

llm = ChatOpenAI(model='gpt-4o')

embedding = OpenAIEmbeddings(model='text-embedding-3-large')


vectorstore = Chroma(collection_name='chroma_stock_news_2', persist_directory="./chroma_3-large", embedding_function=embedding)

retriever = vectorstore.as_retriever(search_kwargs={'k': 100})

system_prompt = (
"""당신은 최신 한국 주식 시장 동향을 분석하고 제공하는 친절한 AI 어시스턴트입니다. 
당신의 임무는 주어진 문맥(context)을 바탕으로 한국 주식 시장에 대한 동향을 전달하는 것입니다.
최신 한국 주식 시장 데이터를 기반으로 단계별로 논리적인 사고 과정을 통해 분석을 진행하고 결론을 도출하세요. 
검색된 다음 문맥(context)을 사용하여 최신 주식 시장 동향을 요약해 주세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.
더 깊게 생각해주세요.

#문맥(Context): 
{context} 

#주식 시장 동향 요약:"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [711]:
trend_result = rag_chain.invoke({"input" : "주식 시장 동향에 대해 알려줘"})

In [712]:
trend_result

{'input': '주식 시장 동향에 대해 알려줘',
 'context': [Document(metadata={'main_category': '미디어 서비스', 'stock_name': '삼성출판사', 'sub_category': '서비스업', 'title': '"한강 노벨문학상 수상, 삼성출판사·예스24 등 급등"'}, page_content='■ 방송 : [CBS매거진] 광주CBS 라디오 1FM 103.1MHz (월~금, 16:30~17:30)■ 제작 : 조성우 PD, 이향미 작가■ 진행 : 정정섭 아나운서■ 방송 일자 : 2024년 10월 14일(월)핵심요약[주식이 알고싶다]DB금융투자 광주지점, "지난주 반도체, 2차전지 등 외국인·기관 매수세"한국은행 3년 2개월 만 기준금리 0.25% 포인트 인하HLB, 미국 FDA 간암 신약 리보세라닙 승인 내년으로 미뤄져 주가 급락DB금융투자 광주지점 조명은 PB.DB금융투자 제공[다음은 DB금융투자 광주지점 조명은 PB'),
  Document(metadata={'main_category': '운송 및 물류', 'stock_name': 'HLB글로벌', 'sub_category': '유통업', 'title': '"한강 노벨문학상 수상, 삼성출판사·예스24 등 급등"'}, page_content='■ 방송 : [CBS매거진] 광주CBS 라디오 1FM 103.1MHz (월~금, 16:30~17:30)■ 제작 : 조성우 PD, 이향미 작가■ 진행 : 정정섭 아나운서■ 방송 일자 : 2024년 10월 14일(월)핵심요약[주식이 알고싶다]DB금융투자 광주지점, "지난주 반도체, 2차전지 등 외국인·기관 매수세"한국은행 3년 2개월 만 기준금리 0.25% 포인트 인하HLB, 미국 FDA 간암 신약 리보세라닙 승인 내년으로 미뤄져 주가 급락DB금융투자 광주지점 조명은 PB.DB금융투자 제공[다음은 DB금융투자 광주지점 조명은 PB'),
  Document(metadata={'main_category': '미디어 서비스', 

In [713]:
trend_result['answer']

"최근 한국 주식 시장에서는 다양한 요인들이 복합적으로 작용하고 있습니다. 먼저, 한국은행이 3년 2개월 만에 기준금리를 0.25% 포인트 인하하면서 금리 인하에 따른 시장 유동성 증가가 예상되었으나, 외국인 투자자들의 국내 주식 매도가 지속되고 있어 시장의 활력을 잃고 있는 모습입니다.\n\n특히, 반도체와 2차 전지 관련 주식에서 외국인과 기관투자자의 매수세가 관찰되었지만, 전반적으로 국내 증시는 박스권에서 벗어나지 못하고 있습니다. 한국 증시의 유동성 부족과 금투세 불확실성 등의 요인으로 인해 투자자들은 미국 증시로 눈을 돌리는 경향을 보이고 있습니다.\n\n테마주에서도 변동성이 큰 모습을 보이고 있는데, 예를 들어 배우 이정재가 3자배정 유상증자에 참여한다는 소식으로 와이더플래닛의 주가가 급등하기도 했습니다. 이는 단기적으로 테마주에 대한 투자자들의 관심이 높은 것을 보여주지만, 기업 실적과는 무관한 인맥 중심의 투자로 '폭탄 돌리기' 위험성이 있다는 우려도 나타나고 있습니다.\n\n전반적으로 시장은 관망세를 보이며, 투자자들은 미국 연방공개시장위원회(FOMC)의 결과 발표 등 주요 경제 일정에 주목하고 있습니다. 이러한 상황 속에서 코스피는 2500~2800포인트, 코스닥은 740~820포인트의 박스권 장세가 예상되며, 실적 성장과 모멘텀이 좋은 업종과 종목군에서의 대응이 필요하다는 전략이 제시되고 있습니다.\n\n마지막으로, HLB의 간암 신약 리보세라닙 승인 연기로 인해 주가가 급락하는 등 개별 기업의 이슈도 주가에 큰 영향을 미치고 있는 상황입니다. 투자자들은 이러한 변동성 속에서 신중한 접근이 필요할 것으로 보입니다."

In [ ]:
def number_and_parse_result_all_rows(data):
    parsed_data = []
    
    for idx, entry in enumerate(data):
        user_info = {
            '사용자 선호 카테고리': entry[0],
            '투자 성향': entry[1],
            '투자 목표': entry[2],
            '선호하는 투자 전략': entry[3],
            '선호 섹터': entry[4],
            '리스크 관리': entry[5],
        }
        id = str(idx).zfill(2) 
        parsed_data.append((id, user_info))
    
    return parsed_data

In [ ]:
from sqlalchemy import asc

def get_category_and_investment_cartesian():

  engine = create_engine(f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_SCHEME}')

  Session = sessionmaker(bind=engine)
  session = Session()

  metadata = MetaData()

  IT_table = Table('INVESTMENT_TYPE', metadata, autoload_with=engine)
  main_category_table = Table('MAIN_CATEGORY', metadata, autoload_with=engine)
  stmt = select(
      main_category_table.c.MAIN_CATEGORY_NAME,
      IT_table.c.INVESTMENT_TYPE_NAME,
      IT_table.c.CONTENT,
      IT_table.c.STOCK_ALLOCATION_CONTENT,
      IT_table.c.STOCK_SELECTION_CONTENT,
      IT_table.c.RISK_MANAGEMENT_CONTENT,
  ).order_by(
    asc(main_category_table.c.MAIN_CATEGORY_ID), 
    asc(IT_table.c.INVESTMENT_TYPE_ID)
  )

  result = session.execute(stmt)

  recommend_list = [(row[0], row[1], row[2], row[3], row[4],row[5]) for row in result]
  session.close()

  return recommend_list

In [ ]:
recommend_list = get_category_and_investment_cartesian()

C:\Users\student\AppData\Local\Temp\ipykernel_18884\2810361819.py:26: SAWarning: SELECT statement has a cartesian product between FROM element(s) "INVESTMENT_TYPE" and FROM element "MAIN_CATEGORY".  Apply join condition(s) between each element to resolve.
  result = session.execute(stmt)


In [ ]:
def number_and_parse_result_all_rows(data):
    parsed_data = []
    
    for idx, entry in enumerate(data):
        user_info = {
            '사용자 선호 카테고리': entry[0],
            '투자 성향': entry[1],
            '투자 목표': entry[2],
            '선호하는 투자 전략': entry[3],
            '선호 섹터': entry[4],
            '리스크 관리': entry[5],
        }
        id = str(idx).zfill(2) 
        parsed_data.append((id, user_info))
    
    return parsed_data

In [ ]:
db_category_investment_data = get_category_and_investment_cartesian()
db_category_investment_data_with_num = number_and_parse_result_all_rows(db_category_investment_data)


C:\Users\student\AppData\Local\Temp\ipykernel_18884\2810361819.py:26: SAWarning: SELECT statement has a cartesian product between FROM element(s) "MAIN_CATEGORY" and FROM element "INVESTMENT_TYPE".  Apply join condition(s) between each element to resolve.
  result = session.execute(stmt)


In [ ]:
db_category_investment_data_with_num

[('00',
  {'사용자 선호 카테고리': '숙박 및 음식',
   '투자 성향': '안정형',
   '투자 목표': '예금이나 적금 수준의 수익률을 기대하며, 투자원금에 손실이 발생하는 것을 원하지 않습니다.',
   '선호하는 투자 전략': '주식 투자 비중은 10% 이하로 설정합니다. 대부분의 자산은 안정적인 금융 상품(CMA, MMF)에 투자하고, 주식은 극히 소량만 투자하는 것이 적합합니다.',
   '선호 섹터': '안정적이고 배당 수익이 높은 대형 우량주를 선택합니다. 주가 변동이 비교적 적고, 꾸준한 현금 흐름을 창출하는 기업에 투자하는 것이 좋습니다. (예: 공기업, 필수소비재 기업)',
   '리스크 관리': '원금 손실을 방지하기 위해 투자 금액을 극히 소량으로 유지하고, 장기적으로 안정적인 수익을 추구합니다. 시장의 변동에 민감하게 대응하지 않고, 장기 보유를 기본 전략으로 합니다.'}),
 ('01',
  {'사용자 선호 카테고리': '숙박 및 음식',
   '투자 성향': '안정추구형',
   '투자 목표': '투자원금의 손실위험은 최소화하고, 이자소득이나 배당소득 수준의 안정적인 투자를 목표로 합니다. 다만 수익을 위해 단기적인 손실을 수용할 수 있으며, 예·적금보다 높은 수익을 위해 자산 중의 일부를 변동성 높은 상품에 투자할 의향이 있습니다',
   '선호하는 투자 전략': '주식에 20~30% 비중을 배정하고, 나머지는 채권형 펀드 또는 장기 회사채펀드에 투자합니다. 주식 비중을 다소 늘리되, 여전히 안전 자산의 비중이 더 높습니다.',
   '선호 섹터': '배당 성향이 높은 대형 우량주나 안정적인 성장세를 보이는 기업을 중심으로 선택합니다. 글로벌 필수소비재, 에너지, 헬스케어와 같은 안정적 섹터에서 종목을 선택하는 것이 좋습니다.',
   '리스크 관리': '주식 시장 변동성에 대비하여 정기적인 포트폴리오 점검과 리밸런싱을 실시합니다. 손실 가능성을 낮추기 위해 투자 금액의 일부는 안전 자산에 분산합니다.'}),
 ('

In [729]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

def set_recommend_stock_RAG(prompt_type):
  llm = ChatOpenAI(model='gpt-4o')

  embedding = OpenAIEmbeddings(model='text-embedding-3-large')

  vectorstore = Chroma(collection_name='chroma_stock_news_2', persist_directory="./chroma_3-large", embedding_function=embedding)
  retriever = vectorstore.as_retriever(search_kwargs={'k': 15})

  system_prompt = ()
  if(prompt_type == "recommend") :
    system_prompt = (
      """당신은 사용자에게 맞춤형 종목 추천을 제공하는 한국 금융 전문가 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)을 바탕으로 사용자의 투자 성향과 선호하는 섹터에 대한 정보를 제공하는 것입니다. 
    당신의 역할은 사용자가 제공한 정보에 맞춰, 최신 뉴스 데이터를 바탕으로 해당 사용자에게 적합한 종목을 추천하는 것입니다. 
    뉴스 데이터는 관련 섹터의 최근 트렌드와 성과를 반영해야 하며, 사용자의 투자 전략과 리스크 성향에 맞는 리포트를 만들어주세요
    코스피, 코스닥 종목을 기반으로 종목을 추천해주세요.
    만약, 주어진 문맥(context)에서 동향을 찾을 수 없다면, `주어진 정보에서 시장 동향에 대한 정보를 찾을 수 없습니다`라고 답하세요.

    사용자 정보는 다음과 같습니다.
    투자 성향: 
    투자 목표: 
    선호하는 투자 전략:
    선호 섹터:
    리스크 관리:
    사용자 선호 카테고리:
    위 정보를 바탕으로, 사용자의 투자 성향과 최근 뉴스 트렌드에 맞는 한국 종목을 추천해 주세요. 당신의 임무는 주어진 문맥(context)을 바탕으로 뉴스의 최근 동향과 관련 섹터에서 주목할 만한 최신 정보를 제공하는 것입니다.

    #사용자정보(User Info): 
    {context} 

    #추천종목:"""
    )
  elif(prompt_type == "trend"):
    system_prompt = (
    """당신은 사용자에게 전달받은 정보를 토대로 사용자가 관심있어 하는 금융 시장 동향을 제공하는 전문 어시스턴트입니다. 
    당신의 임무는 주어진 문맥(context)을 바탕으로 금융 시장의 최근 동향에서 사용자가 주목할 만한 정보를 제공하는 것입니다.
    뉴스 데이터는 관련 섹터의 최근 트렌드와 성과를 반영해야 하며, 사용자의 투자 전략과 리스크 성향에 맞는 뉴스를 제안해야 합니다.
    사용자가 제공한 정보에 맞춰, 최신 뉴스 데이터를 바탕으로 해당 사용자에게 주식 시장 트렌드 리포트를 제공해주세요 
    
    사용자 정보는 다음과 같습니다.
    투자 성향: 
    투자 목표: 
    선호하는 투자 전략:
    선호 섹터:
    리스크 관리:
    사용자 선호 카테고리:
    위의 사용자 정보를 바탕으로, 사용자가 관심있어 할만한 뉴스 동향 리포트를 제공해주세요.

    ####문맥(context)
    {context}
    
    ### 작업 흐름:
    1. **문맥(context)**: 제공된 문맥 뉴스 데이터를 분석하여 시장에서 주목할 만한 트렌드를 파악합니다.
    2. **사용자 정보 분석**: 입력된 사용자 정보를 기반으로, 사용자가 선호하는 투자 전략과 카테고리에 맞춰 맞춤형 뉴스 요약를 제공합니다.
    3. **리스크 관리 반영**: 사용자의 리스크 관리 성향을 반영하여 적절한 전략을 추천합니다.

    ### 제공할 내용:
    리포트 제목 : 생성한 리포트의 제목
    사용자 맞춤형 동향: 사용자가 관심을 가질만한 관련 뉴스 동향 요약
    카테고리 성과: 사용자가 선호하는 카테고리의 최신 성과 및 주목할만한 트렌드
    리스크 관리 전략 반영: 사용자 리스크 관리 전략을 고려한 맞춤형 정보 제공"""
    )

  prompt = ChatPromptTemplate.from_messages(
      [
          ("system", system_prompt),
          ("human", "{input}"),
      ]
  )

  question_answer_chain = create_stuff_documents_chain(llm, prompt)
  rag_chain = create_retrieval_chain(retriever, question_answer_chain)

  return rag_chain

def make_recommend_stock_answer_AI(recommend_list_data, type):
  rag_chain = set_recommend_stock_RAG(type)
  answer_list = []
  for entry in recommend_list_data:
    id = entry[0]
    input = '\n'.join([f"{key}: {value}" for key, value in entry[1].items()])
    result = rag_chain.invoke({"input" : input})
    answer = result['answer']
    answer_list.append((id, answer))

  return answer_list

answer_list = make_recommend_stock_answer_AI(db_category_investment_data_with_num, "trend")

In [730]:
answer_list 

[('00',
  '리포트 제목: "안정형 투자자를 위한 숙박 및 음식 섹터의 최신 동향과 전략"\n\n사용자 맞춤형 동향: 최근 숙박 및 음식 섹터에서는 코로나19 이후 여행 수요가 회복되면서 관련 기업들이 주목받고 있습니다. 특히, 대형 호텔 체인과 글로벌 음식 프랜차이즈 기업들이 안정적인 운영을 통해 꾸준한 배당을 제공하고 있습니다. 이러한 기업들은 경제 회복기에도 안정적인 수익 흐름을 유지할 가능성이 높아, 장기 보유에 적합한 투자 대상으로 평가됩니다.\n\n카테고리 성과: 숙박 및 음식 업종의 대표적인 대형 우량주들은 최근 몇 년간 안정적인 배당 성과를 보이고 있습니다. 예를 들어, 글로벌 호텔 체인들은 여행 회복세에 힘입어 수익성이 점차 개선되고 있으며, 일부 기업은 배당금을 인상하고 있습니다. 필수소비재 부분에서도 식음료 대기업들이 안정적인 수익과 배당 성과를 유지하고 있습니다.\n\n리스크 관리 전략 반영: 사용자의 리스크 관리 성향에 맞춰, 투자 금액을 극히 소량으로 유지하고 안정적인 수익을 추구하는 전략을 추천합니다. 현재 시장에서는 공기업이나 필수소비재 기업의 배당 성향이 높은 종목에 주목할 필요가 있습니다. 이러한 기업들은 경제 불확실성 속에서도 꾸준한 현금 흐름을 유지하고 있어 장기적인 관점에서 안정적인 투자가 가능합니다. 주식 비중을 10% 이하로 유지하며, 나머지 자산은 CMA나 MMF와 같은 안정적인 금융 상품에 투자하여 원금 손실을 방지하는 것이 좋습니다.'),
 ('01',
  '리포트 제목: "안정적인 수익을 위한 숙박 및 음식 섹터의 투자 기회"\n\n사용자 맞춤형 동향:\n최근 숙박 및 음식 섹터는 팬데믹 이후 회복세를 보이며 투자자들의 주목을 받고 있습니다. 특히, 글로벌 여행 수요의 증가와 함께 호텔 및 레스토랑 산업이 다시 붐을 이루고 있습니다. 이와 더불어, 대형 호텔 체인과 레스토랑 기업들이 주식 시장에서 안정적인 성장을 보이고 있으며, 배당 성향이 높은 기업들은 안정적인 수익을 제공할 가능성이 큽니다.\n\n카테

In [720]:
## 제목 추출
import re
# 첫 번째 : 이후 첫 번째 \n까지의 타이틀을 추출하는 함수
def extract_report_titles(data):
    titles = []
    for _, text in data:
        # 첫 번째 : 이후에 오는 텍스트를 추출하고, 첫 번째 \n 이전까지의 텍스트를 가져옴
        match = re.search(r':\s*(.*?)\n', text)
        if match:
            titles.append(match.group(1).strip())
        else:
            titles.append("타이틀 없음")
    return titles

titles = extract_report_titles(answer_list)

result_title = [title.replace('"', '').replace('#', '') for title in titles]

print(result_title)

['안정적인 수익을 위한 숙박 및 음식 섹터 투자 동향', '안정적 수익을 위한 숙박 및 음식 섹터 투자 동향 보고서', '중위험 중수익 투자자를 위한 숙박 및 음식 업종의 최신 동향', '**혁신 섹터와 글로벌 분산 투자 전략으로 본 숙박 및 음식업 동향**', '공격적 투자자들을 위한 숙박 및 음식 산업의 최신 동향과 전략적 기회', '안정적 수익을 위한 건설 및 에너지 부문 투자 전략', '안정성과 성장의 조화: 건설 및 에너지 섹터의 최신 동향', '건설 및 에너지 섹터 중심의 중위험 중수익 투자 전략', '혁신 섹터와 에너지 중심의 고수익 투자 전략: 최신 시장 동향 및 기회', '에너지 및 건설 섹터에서의 기회 탐색: 변동성을 활용한 공격적 투자 전략', '안정형 투자자를 위한 금융 및 보험 시장 동향 리포트', '안정적인 수익을 위한 금융 및 보험 시장의 최신 동향**', '중위험 중수익 투자자를 위한 최신 금융 및 보험 시장 동향', '** 혁신 섹터와 글로벌 시장에서 기회를 찾는 금융 및 보험 동향', '공격 투자자들을 위한 금융 시장 최신 동향 및 전략', '안정형 투자자를 위한 통신 및 IT 섹터의 최신 동향 및 전략', '안정적 성장과 수익성을 위한 통신 및 IT 부문의 투자 동향 보고서', '** IT 및 통신 부문에서의 성장 기회와 안정성 확보 전략', 'IT 및 통신 섹터를 중심으로 한 고위험 고수익 투자 전략', '** 공격투자형 포트폴리오를 위한 최신 통신 및 IT 섹터 트렌드', '운송 및 물류 섹터에서의 안정적 투자 기회', '안정성과 성장의 균형: 운송 및 물류 섹터에서의 투자 기회', '운송 및 물류 섹터에서의 중위험 중수익 투자 전략', '운송 및 물류 시장에서의 혁신과 성장 기회 탐색', '고수익을 노리는 공격투자자를 위한 운송 및 물류 섹터의 최신 동향', '안정적 수익을 위한 농업 및 자원 섹터의 최신 투자 동향', '안정적 성장을 위한 농업 및 자원 섹터 투자 전략', '중위험 중수익을 목표로 한 농업 및 자원 섹

In [721]:
def remove_text_after_double_newline(text):
    # 처음 나오는 \n\n 뒤의 모든 문자를 제거
    return re.sub(r'.*\n\n', '', text)

# 데이터를 처리하여 \n\n 뒤의 모든 문자 제거
result_content = [(item[0], remove_text_after_double_newline(item[1])) for item in answer_list]


print(result_content)

result_content

[('00', '#### 사용자 맞춤형 동향\n#### 카테고리 성과\n#### 리스크 관리 전략 반영\n귀하의 안정적인 투자 성향을 고려할 때, 주식 투자 비중을 10% 이하로 유지하면서 나머지 자산을 CMA, MMF와 같은 안정적인 금융 상품에 투자하는 것이 이상적입니다. 주식 투자는 대형 우량주 중에서도 필수소비재 기업이나 공기업을 중심으로 선택하는 것이 좋습니다. 이러한 기업들은 주가 변동성이 상대적으로 낮고, 꾸준한 배당을 통해 안정적인 수익을 제공할 수 있습니다. 장기 보유를 통해 시장 변동에 대한 민감도를 낮추고, 원금 손실을 방지할 수 있는 전략을 추천합니다.'), ('01', '사용자 맞춤형 동향:\n카테고리 성과:\n리스크 관리 전략 반영:\n귀하의 안정추구형 투자 성향과 리스크 관리를 고려하여, 주식 비중을 20~30%로 유지하면서도 안정적인 배당 수익을 제공하는 대형 우량주에 집중할 것을 추천합니다. 나머지 자산은 채권형 펀드와 장기 회사채 펀드에 투자하여 변동성을 낮추고 안정성을 강화할 수 있습니다. 주식 시장 변동성에 대비해 정기적인 포트폴리오 점검과 리밸런싱을 권장하며, 특히 경제 상황 변화에 민감한 숙박 및 음식 섹터의 특성을 고려해 시장 상황에 맞는 적시 대응이 중요합니다.'), ('02', '**사용자 맞춤형 동향**:\n**카테고리 성과**:\n**리스크 관리 전략 반영**:\n사용자의 리스크 관리 전략을 고려하여, 현재의 시장 회복세에 맞춰 숙박 및 음식 업종 내의 중간 규모 성장 기업에 대한 펀드 투자 비중을 늘리는 것이 바람직할 수 있습니다. 그러나, 큰 하락장이 발생할 경우에는 기존 투자 비중을 조정하여 주식 비중을 줄이고, 채권 및 적립식 펀드로의 전환을 통해 포트폴리오의 변동성을 낮추는 전략이 유효할 것입니다. 분산 투자와 정기적인 투자 점검을 통해 리스크를 지속적으로 관리할 수 있습니다.'), ('03', '사용자 맞춤형 동향:\n카테고리 성과:\n리스크 관리 전략 반영:\n적극적인 투자 성향을 가진 사용자를 위해

[('00',
  '#### 사용자 맞춤형 동향\n#### 카테고리 성과\n#### 리스크 관리 전략 반영\n귀하의 안정적인 투자 성향을 고려할 때, 주식 투자 비중을 10% 이하로 유지하면서 나머지 자산을 CMA, MMF와 같은 안정적인 금융 상품에 투자하는 것이 이상적입니다. 주식 투자는 대형 우량주 중에서도 필수소비재 기업이나 공기업을 중심으로 선택하는 것이 좋습니다. 이러한 기업들은 주가 변동성이 상대적으로 낮고, 꾸준한 배당을 통해 안정적인 수익을 제공할 수 있습니다. 장기 보유를 통해 시장 변동에 대한 민감도를 낮추고, 원금 손실을 방지할 수 있는 전략을 추천합니다.'),
 ('01',
  '사용자 맞춤형 동향:\n카테고리 성과:\n리스크 관리 전략 반영:\n귀하의 안정추구형 투자 성향과 리스크 관리를 고려하여, 주식 비중을 20~30%로 유지하면서도 안정적인 배당 수익을 제공하는 대형 우량주에 집중할 것을 추천합니다. 나머지 자산은 채권형 펀드와 장기 회사채 펀드에 투자하여 변동성을 낮추고 안정성을 강화할 수 있습니다. 주식 시장 변동성에 대비해 정기적인 포트폴리오 점검과 리밸런싱을 권장하며, 특히 경제 상황 변화에 민감한 숙박 및 음식 섹터의 특성을 고려해 시장 상황에 맞는 적시 대응이 중요합니다.'),
 ('02',
  '**사용자 맞춤형 동향**:\n**카테고리 성과**:\n**리스크 관리 전략 반영**:\n사용자의 리스크 관리 전략을 고려하여, 현재의 시장 회복세에 맞춰 숙박 및 음식 업종 내의 중간 규모 성장 기업에 대한 펀드 투자 비중을 늘리는 것이 바람직할 수 있습니다. 그러나, 큰 하락장이 발생할 경우에는 기존 투자 비중을 조정하여 주식 비중을 줄이고, 채권 및 적립식 펀드로의 전환을 통해 포트폴리오의 변동성을 낮추는 전략이 유효할 것입니다. 분산 투자와 정기적인 투자 점검을 통해 리스크를 지속적으로 관리할 수 있습니다.'),
 ('03',
  '사용자 맞춤형 동향:\n카테고리 성과:\n리스크 관리 전략 반영:\n적극적인 투자 성향을

In [722]:
answer_list

[('00',
  '### 리포트 제목: 안정적인 수익을 위한 숙박 및 음식 섹터 투자 동향\n\n#### 사용자 맞춤형 동향\n최근 숙박 및 음식 섹터는 포스트 팬데믹 회복과 함께 안정적인 성장세를 보이고 있습니다. 특히, 대형 호텔 체인 및 글로벌 식품 기업들은 지속적인 배당을 제공하면서 투자자들 사이에서 안정적인 수익원으로 주목받고 있습니다. 예를 들어, 일부 글로벌 호텔 체인은 팬데믹 이후 빠른 회복세를 보이며 안정적인 배당 수익을 유지하고 있습니다. 또한, 식품 기업들은 지속적인 수요와 함께 꾸준한 매출 성장을 기록하고 있어 투자자들에게 긍정적인 신호를 주고 있습니다.\n\n#### 카테고리 성과\n숙박 및 음식 섹터의 주요 대형 우량주들은 최근 몇 분기 동안 안정적인 배당을 제공하며 투자자들의 관심을 끌고 있습니다. 특히, 필수소비재 기업들은 경제 불확실성에도 불구하고 꾸준한 성장을 이어가고 있습니다. 이러한 기업들은 경쟁력 있는 배당 수익률을 제공하며, 주가 변동성이 낮아 안정적인 현금 흐름을 창출하고 있습니다.\n\n#### 리스크 관리 전략 반영\n귀하의 안정적인 투자 성향을 고려할 때, 주식 투자 비중을 10% 이하로 유지하면서 나머지 자산을 CMA, MMF와 같은 안정적인 금융 상품에 투자하는 것이 이상적입니다. 주식 투자는 대형 우량주 중에서도 필수소비재 기업이나 공기업을 중심으로 선택하는 것이 좋습니다. 이러한 기업들은 주가 변동성이 상대적으로 낮고, 꾸준한 배당을 통해 안정적인 수익을 제공할 수 있습니다. 장기 보유를 통해 시장 변동에 대한 민감도를 낮추고, 원금 손실을 방지할 수 있는 전략을 추천합니다.'),
 ('01',
  '리포트 제목: "안정적 수익을 위한 숙박 및 음식 섹터 투자 동향 보고서"\n\n사용자 맞춤형 동향:\n최근 숙박 및 음식 섹터는 글로벌 경제 회복과 여행 수요 증가로 인해 긍정적인 성장세를 보이고 있습니다. 주요 호텔 체인과 레스토랑 프랜차이즈는 팬데믹 이후 회복세를 이어가며 고객 유입이 증가하고 있습니다.

In [453]:
import pandas as pd

df = pd.DataFrame(answer_list)
df.to_csv("recommend_report_data.csv", index=False, encoding='euc-kr')

In [437]:
answer_list = make_recommend_stock_answer_AI([('00',
  {'사용자 선호 카테고리': '숙박 및 음식',
   '투자 성향': '공격투자형',
   '투자 목표': '시장평균수익률을 훨씬 넘어서는 높은 수준의 투자수익을 추구하며, 이를 위해 자산가치의 변동에 따른 손실위험을 적극 수용할 수 있습니다. 투자자금 대부분을 주식, 주식형펀드 또는 파생상품 등의 위험자산에 투자할 의향이 있습니다.',
   '선호하는 투자 전략': '주식 비중을 70% 이상으로 설정하고, 나머지는 주식형 펀드 또는 파생상품에 투자합니다. 자산의 10% 정도는 직접 주식 투자로 활용하며, 주식에 대부분의 자산을 배정합니다.',
   '선호 섹터': '고수익을 기대할 수 있는 중소형 성장주와 신흥 시장에 투자합니다. 높은 변동성을 감수하고 급성장 가능성이 있는 기술주, 바이오주, 에너지주 등 적극적으로 성장 가능성이 높은 섹터에 투자합니다.',
   '리스크 관리': '시장 변화에 빠르게 대응할 수 있는 능동적인 매매 전략을 구사합니다. 손실을 감수할 준비를 하되, 적극적인 리스크 관리를 위해 헤지 전략(옵션, 선물 등)을 활용할 수 있습니다.'})], "trend")

트렌드 실행


In [438]:
answer_list

[('00',
  '## 사용자 맞춤형 동향\n\n### 숙박 및 음식 섹터 관련 뉴스 동향\n최근 숙박 및 음식 섹터에서는 포스트 팬데믹 상황에서의 회복과 변화를 위한 여러 움직임이 관측되고 있습니다. 특히, 디지털 전환을 통한 예약 시스템 개선, 고객 맞춤형 서비스 제공, 그리고 지속 가능한 식재료 사용에 대한 노력이 두드러지고 있습니다. 이러한 변화는 기술적 진보와 함께 고객 경험을 강화하고, 운영 효율성을 높이는 방향으로 나아가고 있습니다.\n\n### 카테고리 성과\n숙박 및 음식 업종은 최근 경제 회복과 함께 성장이 기대되는 섹터 중 하나입니다. 특히, 디지털 트랜스포메이션을 도입한 기업들은 경쟁력을 높이며 시장 내에서 주목받고 있습니다. 또한, 배달 서비스의 증가와 함께 음식업계의 혁신적인 변화는 높은 성장 가능성을 보입니다.\n\n## 리스크 관리 전략 반영\n귀하의 공격적인 투자 성향과 선호하는 리스크 관리 방식에 따라, 다음과 같은 전략을 추천드립니다:\n\n1. **고성장 중소형주 및 신흥 시장 투자**: 숙박 및 음식 섹터 내에서도 기술력을 바탕으로 급성장하는 중소형주에 주목합니다. 특히, 신흥 시장에서의 확장 가능성을 가진 기업을 분석하여 투자 기회를 모색할 수 있습니다.\n\n2. **헤지 전략 활용**: 변동성이 높은 시장 상황을 대비해 옵션이나 선물 등을 활용한 헤지 전략을 병행하여 리스크를 관리합니다.\n\n3. **기술 및 디지털 전환 기업 집중**: 디지털 혁신을 선도하는 기업들에 대한 투자를 늘림으로써, 기술 발전에 따른 추가적인 수익을 기대할 수 있습니다.\n\n4. **시장 변화에 대한 빠른 대응**: 시장 변화에 민첩하게 대응할 수 있도록, 주기적으로 포트폴리오를 재조정하고 최신 시장 동향을 지속적으로 모니터링합니다.\n\n이러한 전략은 귀하의 투자 목표와 성향에 부합하며, 시장 평균을 초과하는 수익률을 추구하는 데 기여할 수 있습니다.')]

In [429]:

result = rag_chain.invoke({"input" : """투자 성향: 적극투자형
투자 목표: 투자원금의 보전보다는 위험을 감내하더라도 높은 수준의 투자수익을 추구합니다. 투자자금의 상당 부분을 주식, 주식형펀드* 또는 파생상품 등의 위험자산에 투자할 의향이 있습니다.
선호하는 투자 전략: 주식에 60~70% 비중을 두고, 나머지는 고위험 상품(주식형 펀드, ELS 등)에 투자합니다. 주식 투자의 비중을 높게 가져가며, 해외 주식에도 적극적으로 투자합니다.
선호 섹터: 성장 가능성이 높은 IT, 바이오, 신재생에너지와 같은 혁신 섹터에 집중합니다. 국내외 시장에서 트렌드에 맞는 성장주나 기술주를 선택하며, 고수익을 기대할 수 있는 중소형주도 고려합니다.
리스크 관리: 주식시장 하락기에 대비하여 손절매 기준을 미리 설정하고, 빠르게 대응할 수 있는 전략을 마련합니다. 또한, 포트폴리오 내에서 다양한 국가 및 산업에 분산 투자하여 리스크를 분산합니다.
사용자 선호 카테고리: 숙박 및 음식"""})


In [400]:
result['answer']

'사용자의 투자 성향과 선호 섹터에 맞춰, 최근 뉴스 데이터를 바탕으로 적합한 한국 종목을 추천해드리겠습니다.\n\n1. **IT 섹터**:\n   - **삼성전자 (005930.KS)**: 세계적인 반도체 및 전자제품 제조업체로, 최근 AI 및 반도체 수요 증가와 관련된 긍정적인 뉴스가 많습니다. 글로벌 시장에서의 경쟁력을 바탕으로 장기적인 성장 가능성이 높습니다.\n\n2. **바이오 섹터**:\n   - **셀트리온 (068270.KS)**: 바이오의약품 개발 및 제조업체로, 최근 글로벌 바이오시밀러 시장에서의 성과가 주목받고 있습니다. 혁신적인 연구개발로 미래 성장성을 기대할 수 있습니다.\n\n3. **신재생에너지 섹터**:\n   - **한화솔루션 (009830.KS)**: 태양광 에너지 관련 사업을 확장하고 있으며, 글로벌 에너지 전환 트렌드에 맞춰 성장 중입니다. 정부의 그린 정책과 맞물려 수혜가 예상됩니다.\n\n4. **중소형 성장주**:\n   - **펄어비스 (263750.KQ)**: 게임 개발업체로, 혁신적인 게임 콘텐츠와 글로벌 시장 확장 전략이 주목받고 있습니다. 최근 게임 산업의 성장세를 고려할 때 유망한 중소형주로 평가됩니다.\n\n5. **숙박 및 음식 섹터**:\n   - **호텔신라 (008770.KS)**: 숙박과 면세점 비즈니스를 운영하는 기업으로, 최근 여행 및 관광 산업 회복에 따른 수혜가 기대됩니다. 글로벌 관광객 증가와 국내 외식산업의 회복세가 긍정적입니다.\n\n이 종목들은 사용자의 투자 전략 및 섹터 선호도에 부합하며, 최근의 산업 트렌드와 성장 가능성을 반영하고 있습니다. 투자 시, 개별 종목의 리스크와 시장 변동성을 고려하여 포트폴리오를 구성하시기 바랍니다.'

In [211]:
def parse_user_data(data):
    parsed_data = []
    
    for entry in data:
        user_info = {
            '투자 성향': entry[1],
            '투자 목표': entry[2],
            '선호하는 투자 전략': entry[3],
            '선호 섹터': entry[4],
            '리스크 관리': entry[5],
            '사용자 선호 카테고리': entry[6]
        }
        parsed_data.append(user_info)
    
    return parsed_data

# 파싱된 데이터 출력
parsed_user_data = parse_user_data(recommend_list)

for user in parsed_user_data:
    print(f"투자 성향: {user['투자 성향']}")
    print(f"투자 목표: {user['투자 목표']}")
    print(f"선호하는 투자 전략: {user['선호하는 투자 전략']}")
    print(f"선호 섹터: {user['선호 섹터']}")
    print(f"리스크 관리: {user['리스크 관리']}")
    print(f"사용자 선호 카테고리: {user['사용자 선호 카테고리']}")
    print("\n" + "-"*50 + "\n")

투자 성향: 적극투자형
투자 목표: 투자원금의 보전보다는 위험을 감내하더라도 높은 수준의 투자수익을 추구합니다. 투자자금의 상당 부분을 주식, 주식형펀드* 또는 파생상품 등의 위험자산에 투자할 의향이 있습니다.
선호하는 투자 전략: 주식에 60~70% 비중을 두고, 나머지는 고위험 상품(주식형 펀드, ELS 등)에 투자합니다. 주식 투자의 비중을 높게 가져가며, 해외 주식에도 적극적으로 투자합니다.
선호 섹터: 성장 가능성이 높은 IT, 바이오, 신재생에너지와 같은 혁신 섹터에 집중합니다. 국내외 시장에서 트렌드에 맞는 성장주나 기술주를 선택하며, 고수익을 기대할 수 있는 중소형주도 고려합니다.
리스크 관리: 주식시장 하락기에 대비하여 손절매 기준을 미리 설정하고, 빠르게 대응할 수 있는 전략을 마련합니다. 또한, 포트폴리오 내에서 다양한 국가 및 산업에 분산 투자하여 리스크를 분산합니다.
사용자 선호 카테고리: 숙박 및 음식

--------------------------------------------------

투자 성향: 적극투자형
투자 목표: 투자원금의 보전보다는 위험을 감내하더라도 높은 수준의 투자수익을 추구합니다. 투자자금의 상당 부분을 주식, 주식형펀드* 또는 파생상품 등의 위험자산에 투자할 의향이 있습니다.
선호하는 투자 전략: 주식에 60~70% 비중을 두고, 나머지는 고위험 상품(주식형 펀드, ELS 등)에 투자합니다. 주식 투자의 비중을 높게 가져가며, 해외 주식에도 적극적으로 투자합니다.
선호 섹터: 성장 가능성이 높은 IT, 바이오, 신재생에너지와 같은 혁신 섹터에 집중합니다. 국내외 시장에서 트렌드에 맞는 성장주나 기술주를 선택하며, 고수익을 기대할 수 있는 중소형주도 고려합니다.
리스크 관리: 주식시장 하락기에 대비하여 손절매 기준을 미리 설정하고, 빠르게 대응할 수 있는 전략을 마련합니다. 또한, 포트폴리오 내에서 다양한 국가 및 산업에 분산 투자하여 리스크를 분산합니다.
사용자 선호 카테고리: 건설 및 에너지

-------

In [ ]:
make_recommend_stock_answer_AI(db_category_investment_data)

In [ ]:

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

llm = ChatOpenAI(model='gpt-4o')
# news_data = fetchDBNews()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

vectorstore = Chroma(collection_name='chroma_stock_news', persist_directory="./chroma_3-large", embedding_function=embedding)
# retriever = vectorstore.as_retriever(search_kwargs={'k': 7, 'filter': {'main_category':'숙박 및 음식'}})
# retriever = vectorstore.as_retriever()
retriever = vectorstore.as_retriever(search_kwargs={'k': 7})

system_prompt = (
"""당신은 최신 금융 동향을 분석하고 제공하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)을 바탕으로 금융 시장에 대한 동향을 전달하는 것입니다.
검색된 다음 문맥(context)을 사용하여 최신 금융 동향을 요약해 주세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#문맥(Context): 
{context} 

#한국 금융 시장 동향 요약:"""
)

# system_prompt = (
#   """당신은 주식 종목을 분석하고 제공하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)을 바탕으로 주식 종목에 대한 동향을 전달하는 것입니다.
# 검색된 다음 문맥(context)을 사용하여 최신 주식 종목을 요약해 주세요. 만약, 주어진 문맥(context)에서 동향을 찾을 수 없다면, `주어진 정보에서 주식 종목에 대한 정보를 찾을 수 없습니다`라고 답하세요.
# 한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.


# #문맥(Context): 
# {context} 

# #시장 동향 요약:"""
# )
# system_prompt = (
#   """당신은 주식 종목을 분석하고 제공하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)을 바탕으로 주식 종목에 대한 동향을 전달하는 것입니다.
# 최신 시장 데이터를 기반으로 단계별로 논리적인 사고 과정을 통해 분석을 진행하고 결론을 도출하세요. 검색된 다음 문맥(context)을 사용하여 최신 주식 종목을 요약해 주세요. 
# 만약, 주어진 문맥(context)에서 동향을 찾을 수 없다면, `0`라고 답하세요.
# 한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.
# 서론은 달지말고 본론부터 이야기해주세요.

# 더 깊게 생각해주세요.
# #문맥(Context): 
# {context} 

# #시장 동향 요약:"""
# )
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [513]:
from sqlalchemy import select, func
from sqlalchemy.orm import Session

Session = sessionmaker(bind=engine)
session = Session()

metadata = MetaData()

USER = Table('USER', metadata, autoload_with=engine)

# 쿼리 작성
stmt = select(
    USER.c.USER_TYPE,
    USER.c.USER_ID,
    func.count(USER.c.USER_TYPE).label('USER_COUNT')
).group_by(USER.c.USER_TYPE)

# 쿼리 실행
result = session.execute(stmt)


user_grouped_usertype_results = result.fetchall()
# 세션 닫기
session.close()

In [ ]:
answer_list


In [723]:
# 결과를 병합하는 코드
merged_results = []

# 사용자 ID를 기준으로 병합
for sql_result in user_grouped_usertype_results:
    user_type = sql_result[0]
    user_info = sql_result[1]
    # 대응하는 추천 종목을 찾음
    for title, content in zip(result_title, result_content):
        if content[0] == user_type:
            merged_data = (user_info, trend_result['answer'], title, content[1])
            merged_results.append(merged_data)

In [724]:
merged_results

[('3745237385',
  "최근 한국 주식 시장에서는 다양한 요인들이 복합적으로 작용하고 있습니다. 먼저, 한국은행이 3년 2개월 만에 기준금리를 0.25% 포인트 인하하면서 금리 인하에 따른 시장 유동성 증가가 예상되었으나, 외국인 투자자들의 국내 주식 매도가 지속되고 있어 시장의 활력을 잃고 있는 모습입니다.\n\n특히, 반도체와 2차 전지 관련 주식에서 외국인과 기관투자자의 매수세가 관찰되었지만, 전반적으로 국내 증시는 박스권에서 벗어나지 못하고 있습니다. 한국 증시의 유동성 부족과 금투세 불확실성 등의 요인으로 인해 투자자들은 미국 증시로 눈을 돌리는 경향을 보이고 있습니다.\n\n테마주에서도 변동성이 큰 모습을 보이고 있는데, 예를 들어 배우 이정재가 3자배정 유상증자에 참여한다는 소식으로 와이더플래닛의 주가가 급등하기도 했습니다. 이는 단기적으로 테마주에 대한 투자자들의 관심이 높은 것을 보여주지만, 기업 실적과는 무관한 인맥 중심의 투자로 '폭탄 돌리기' 위험성이 있다는 우려도 나타나고 있습니다.\n\n전반적으로 시장은 관망세를 보이며, 투자자들은 미국 연방공개시장위원회(FOMC)의 결과 발표 등 주요 경제 일정에 주목하고 있습니다. 이러한 상황 속에서 코스피는 2500~2800포인트, 코스닥은 740~820포인트의 박스권 장세가 예상되며, 실적 성장과 모멘텀이 좋은 업종과 종목군에서의 대응이 필요하다는 전략이 제시되고 있습니다.\n\n마지막으로, HLB의 간암 신약 리보세라닙 승인 연기로 인해 주가가 급락하는 등 개별 기업의 이슈도 주가에 큰 영향을 미치고 있는 상황입니다. 투자자들은 이러한 변동성 속에서 신중한 접근이 필요할 것으로 보입니다.",
  '안정적인 수익을 위한 숙박 및 음식 섹터 투자 동향',
  '#### 사용자 맞춤형 동향\n#### 카테고리 성과\n#### 리스크 관리 전략 반영\n귀하의 안정적인 투자 성향을 고려할 때, 주식 투자 비중을 10% 이하로 유지하면서 나머지 자산을 CMA, MMF와 같은 안정적인 금융 상품

In [725]:
from sqlalchemy.orm import Session
from sqlalchemy import Column, Integer, String, Text, ForeignKey, TIMESTAMP, func
from sqlalchemy.ext.declarative import declarative_base

def save_recent_market_trend(recent_market_data):
  engine = create_engine(f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_SCHEME}')
  Base = declarative_base()
  class User(Base):
      __tablename__ = 'USER'

      USER_ID = Column(String(50), primary_key=True)
      
  class DailyReport(Base):
      __tablename__ = 'DAILY_REPORT'

      DAILY_REPORT_ID = Column(Integer, primary_key=True, autoincrement=True)
      USER_ID = Column(String(50), ForeignKey('USER.USER_ID'))
      RECENT_TREND_TITLE = Column(String(50))
      RECENT_TREND_CONTENT = Column(Text)
      STOCK_TREND_TITLE = Column(String(50))
      STOCK_TREND_CONTENT = Column(Text)
      KOSDAQ_PRICE = Column(Integer)
      KOSDAQ_PROFIT_RATE = Column(Integer)
      KOSPI_PRICE = Column(Integer)
      KOSPI_PROFIT_RATE = Column(Integer)
      DAILY_TREND_SUMMARIZED_TITLE = Column(String(20))
      DAILY_TREND_SUMMARIZED_CONTENT = Column(Text)
      CREATED_AT = Column(TIMESTAMP, server_default=func.now())
      UPDATED_AT = Column(TIMESTAMP, onupdate=func.now())
  
  Session = sessionmaker(bind=engine)
  session = Session()

  for userId, recent_trend_content, stock_trend_title, stock_trend_content in recent_market_data:
      new_report = DailyReport(
          USER_ID=userId,
          RECENT_TREND_TITLE="최신 시장 동향",
          RECENT_TREND_CONTENT=recent_trend_content,
          STOCK_TREND_TITLE=stock_trend_title,  
          STOCK_TREND_CONTENT=stock_trend_content
      )
      session.add(new_report)

  session.commit()

  session.close()

In [726]:
save_recent_market_trend(merged_results)

C:\Users\student\AppData\Local\Temp\ipykernel_18884\326759350.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
